In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
user_data=pd.read_csv(r'C:\Users\hp\Desktop\users.csv')
post_data=pd.read_csv(r'C:\Users\hp\Desktop\posts.csv',sep=r'\s*,\s*')
views_data=pd.read_csv(r'C:\Users\hp\Desktop\views.csv')

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 4 columns):
_id          118 non-null object
name         118 non-null object
gender       118 non-null object
academics    118 non-null object
dtypes: object(4)
memory usage: 3.8+ KB


In [4]:
post_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 4 columns):
_id          493 non-null object
title        493 non-null object
category     465 non-null object
post_type    493 non-null object
dtypes: object(4)
memory usage: 15.5+ KB


In [5]:
views_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 3 columns):
user_id      1449 non-null object
post_id      1449 non-null object
timestamp    1449 non-null object
dtypes: object(3)
memory usage: 34.0+ KB


In [6]:
user_data.head()

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [7]:
views_data.head(2)

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z


In [8]:
post_data.head(5)

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [9]:
post_data.category=post_data.category.str[0:-1].str.split('|')

In [10]:
post_data.head(2)

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,[Plant Biotechnolog],blog
1,5d6d39567fa40e1417a4931c,Ml and AI,"[Artificial Intelligence, Machine Learning, In...",blog


In [11]:
post_data.dtypes

_id          object
title        object
category     object
post_type    object
dtype: object

In [12]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [13]:
features=['category','post_type','title']
for feature in features:
    post_data[feature]=post_data[feature].apply(clean_data)

In [14]:
post_data.head(1)

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hellothere,[plantbiotechnolog],blog


In [15]:
def create_soup(x):
    return ''.join(x['title'])+' '+' '.join(x['category'])+' '+''.join(x['post_type'])

In [16]:
post_data['soup']=post_data.apply(create_soup,axis=1)

In [17]:
post_data.soup.head(2)

0                    hellothere plantbiotechnolog blog
1    mlandai artificialintelligence machinelearning...
Name: soup, dtype: object

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(post_data['soup'])

In [19]:
count_matrix.shape

(493, 837)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [21]:
post_data = post_data.reset_index()
indices = pd.Series(post_data.index, index=post_data['_id'])

In [22]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the post that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all posts with that post
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the posts based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 1 most similar posts
    sim_scores = sim_scores[1:2]

    # Get the mpost indices
    post_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar post
    return post_data['_id'].iloc[post_indices]

In [23]:
post_index=input('input post index for which similar post to be recommended')
post=post_data['_id'][int(post_index)]
get_recommendations(post, cosine_sim)

input post index for which similar post to be recommended5


4    5d80dfbc6c53455f896e600e
Name: _id, dtype: object

In [24]:
#Recommend post for given user

In [55]:
views_data.sort_values(by=['user_id','timestamp'],ascending=[True,True],inplace=True)

In [56]:
#pd.value_counts(views_data.user_id)
views_data.head(2)

,user_id,post_id,timestamp
0,5d60098a653a331687083238,5e1029f22a37d20505da2a79,2020-03-26T18:25:45.637Z
1,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,2020-03-26T18:49:14.721Z


In [57]:
views_data=views_data.set_index(np.arange(len(views_data.index)))
views_data.head()

,user_id,post_id,timestamp
0,5d60098a653a331687083238,5e1029f22a37d20505da2a79,2020-03-26T18:25:45.637Z
1,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,2020-03-26T18:49:14.721Z
2,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,2020-03-26T19:00:25.693Z
3,5d60098a653a331687083238,5e7a629ccfc8b713f5ac7d84,2020-03-26T19:01:42.384Z
4,5d60098a653a331687083238,5e7a60edcfc8b713f5ac7d82,2020-03-26T19:03:55.284Z


In [58]:
user_data.head()

,index,_id,name,gender,academics,post,soup
0,0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,"[5e1029f22a37d20505da2a79, 5e7cc074cfc8b713f5a...",male undergraduate 5e1029f22a37d20505da2a79 5e...
1,1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate,"[5e7c78fdcfc8b713f5ac7daa, 5e7c7a44cfc8b713f5a...",male graduate 5e7c78fdcfc8b713f5ac7daa 5e7c7a4...
2,2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate,"[5e7e66e7a3258347b42f213c, 5e773c0bcfc8b713f5a...",male undergraduate 5e7e66e7a3258347b42f213c 5e...
3,3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate,"[5eb2c11210426255a7aaa052, 5eabfaf510426255a7a...",male undergraduate 5eb2c11210426255a7aaa052 5e...
4,4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate,"[5e7304761b24db0468e90ce7, 5e77c625cfc8b713f5a...",male undergraduate 5e7304761b24db0468e90ce7 5e...


In [59]:
def edit_data(x,user_data):
    j=0
    for i in range(0,x.shape[0]):
        if(j>117):
            break
        if(x['user_id'][i]==user_data['_id'][j]):
            user_data['post'][j].append(x['post_id'][i])
        if(i<1448):
            if(x['user_id'][i+1]!=x['user_id'][i]):
                j=j+1
            

In [60]:
user_data['post']=""
user_data['post']=user_data['post'].apply(list)

In [61]:
edit_data(views_data,user_data)

In [62]:
user_data.head(118)

,index,_id,name,gender,academics,post,soup
0,0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,"[5e1029f22a37d20505da2a79, 5e7cc074cfc8b713f5a...",male undergraduate 5e1029f22a37d20505da2a79 5e...
1,1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate,"[5e7c78fdcfc8b713f5ac7daa, 5e7c7a44cfc8b713f5a...",male graduate 5e7c78fdcfc8b713f5ac7daa 5e7c7a4...
2,2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate,"[5e7e66e7a3258347b42f213c, 5e773c0bcfc8b713f5a...",male undergraduate 5e7e66e7a3258347b42f213c 5e...
3,3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate,"[5eb2c11210426255a7aaa052, 5eabfaf510426255a7a...",male undergraduate 5eb2c11210426255a7aaa052 5e...
4,4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate,"[5e7304761b24db0468e90ce7, 5e77c625cfc8b713f5a...",male undergraduate 5e7304761b24db0468e90ce7 5e...
5,5,5d90fc1a641ef364939fb6cc,Boshi Kaviya,female,graduate,[5d9b3514979d5962253c2f90],female graduate 5d9b3514979d5962253c2f90
6,6,5d9a2982979d5962253c2f81,Nischaya Sharma,male,undergraduate,"[5e81be54a3258347b42f221d, 5e767d501b24db0468e...",male undergraduate 5e81be54a3258347b42f221d 5e...
7,7,5de50d768eab6401affbb135,thesocialcomment,male,graduate,"[5e77738bcfc8b713f5ac7b36, 5e773c89cfc8b713f5a...",male graduate 5e77738bcfc8b713f5ac7b36 5e773c8...
8,8,5ded443342a8854bf6eaba9a,MAAZ AHMAD RABBANI,male,undergraduate,[5dea816a42a8854bf6eaba89],male undergraduate 5dea816a42a8854bf6eaba89
9,9,5deeef6142a8854bf6eabab9,siddharth saxena,male,undergraduate,"[5d9b950768671220a1b2b153, 5e947c4aa3258347b42...",male undergraduate 5d9b950768671220a1b2b153 5e...


In [63]:
features=['gender','academics','post']
for feature in features:
    user_data[feature]=user_data[feature].apply(clean_data)

In [64]:
user_data.head(2)

,index,_id,name,gender,academics,post,soup
0,0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,"[5e1029f22a37d20505da2a79, 5e7cc074cfc8b713f5a...",male undergraduate 5e1029f22a37d20505da2a79 5e...
1,1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate,"[5e7c78fdcfc8b713f5ac7daa, 5e7c7a44cfc8b713f5a...",male graduate 5e7c78fdcfc8b713f5ac7daa 5e7c7a4...


In [65]:
def create_soup2(x):
    return ''.join(x['gender'])+' '+''.join(x['academics'])+' '+' '.join(x['post'])

In [66]:
user_data['soup']=user_data.apply(create_soup2,axis=1)

In [67]:
user_data.head(3)

,index,_id,name,gender,academics,post,soup
0,0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,"[5e1029f22a37d20505da2a79, 5e7cc074cfc8b713f5a...",male undergraduate 5e1029f22a37d20505da2a79 5e...
1,1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate,"[5e7c78fdcfc8b713f5ac7daa, 5e7c7a44cfc8b713f5a...",male graduate 5e7c78fdcfc8b713f5ac7daa 5e7c7a4...
2,2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate,"[5e7e66e7a3258347b42f213c, 5e773c0bcfc8b713f5a...",male undergraduate 5e7e66e7a3258347b42f213c 5e...


In [68]:
count2 = CountVectorizer(stop_words='english')
count_matrix2 = count2.fit_transform(user_data['soup'])

In [69]:
cosine_sim2 = cosine_similarity(count_matrix2, count_matrix2)

In [70]:
user_data = user_data.reset_index()
indices = pd.Series(user_data.index, index=user_data['_id'])

In [71]:
def get_sim_user(title, cosine_sim):
    # Get the index of the post that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that post
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the posts based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 1 most similar movies
    sim_scores = sim_scores[1:2]

    # Get the movie indices
    user_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return user_indices[0]

In [72]:
user_index=input("input user index for which post has to be recommneded")
user_id=user_data['_id'][int(user_index)]
sim_index=get_sim_user(user_id, cosine_sim2)

input user index for which post has to be recommneded15


In [73]:
print(sim_index)

85


In [74]:
print(user_index)

15


In [75]:
dif_post = [x for x in user_data['post'][sim_index] if x not in user_data['post'][int(user_index)]]
len(dif_post)

19

In [76]:
indices = pd.Series(post_data.index, index=post_data['_id'])

In [77]:
dif_post_index=[]
for i in dif_post:
    dif_post_index.append(indices[i])

In [78]:
def get_recommendation(title, cosine_sim=cosine_sim,dif_post_index=dif_post_index):
    # Get the index of the post that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that post
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the posts based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 1 most similar movies
    sim_scores = sim_scores[1:100]
    post_indices = [i[0] for i in sim_scores]
    for i in post_indices:
        if i in dif_post_index:
            return i
    

    # Get the movie indices
    sim_scores=sim_scores[1:2]
    post_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return post_data['_id'].iloc[post_indices]

In [79]:

l=len(user_data['post'][int(user_index)])
post_last=user_data['post'][int(user_index)][l-1]
print(str(post_last))


5ec821ddec493f4a2655889e


In [80]:
user_data.head(1)

,level_0,index,_id,name,gender,academics,post,soup
0,0,0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,"[5e1029f22a37d20505da2a79, 5e7cc074cfc8b713f5a...",male undergraduate 5e1029f22a37d20505da2a79 5e...


In [81]:
get_recommendation(str(post_last),cosine_sim,dif_post)

121    5e5bb3eed701ab08af792bfa
Name: _id, dtype: object